In [116]:
import pandas as pd
import re
import numpy as np

In [117]:
df_bc = pd.read_csv('data/comments/bitcoin.csv')
df_dj = pd.read_csv('data/comments/dj.csv')
df_ts = pd.read_csv('data/comments/tesla.csv')

In [118]:
from dateutil import parser
from datetime import date
import re

def convert_date(date_string):
    
    if 'ago' in str(date_string):
        today = date.today()
        return today.strftime("%d-%m-%y")
    
    if 'nan' in str(date_string):
        return
    
    dt = parser.parse(date_string)  
    return dt.strftime("%d-%m-%y")

def convert(df):
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    df['date'] = df['date'].apply(lambda x: convert_date(x))
    
    return df
    

In [119]:
df_bc = convert(df_bc)
df_dj = convert(df_dj)
df_ts = convert(df_ts)

In [120]:
del df_ts['+/-']

In [121]:
df_bc = df_bc.dropna(subset=['comment'])
df_dj = df_dj.dropna(subset=['comment'])
df_ts = df_ts.dropna(subset=['comment'])

In [122]:
df_bcp = pd.read_excel('data/price/BTC.xlsx')
df_djp = pd.read_excel('data/price/DJI.xlsx')
df_tsp = pd.read_excel('data/price/TSLA.xlsx')

In [123]:
df_bc_merged = df_bcp.set_index('Date').join(df_bc.set_index('date'))
df_dj_merged = df_djp.set_index('Date').join(df_dj.set_index('date'))
df_ts_merged = df_tsp.set_index('Date').join(df_ts.set_index('date'))

In [124]:
df_bc_merged = df_bc_merged.dropna(subset=['comment'])
df_dj_merged = df_dj_merged.dropna(subset=['comment'])
df_ts_merged = df_ts_merged.dropna(subset=['comment'])

In [125]:
df_bc_merged.head()

,High,Low,Upper Bound,Lower Bound,comment
01-01-18,13921.530273,12877.669922,17401.912841,9658.252441,Drop now to 8600\r\n
01-01-18,13921.530273,12877.669922,17401.912841,9658.252441,"If a panic kicks in, there is no support forge..."
01-01-18,13921.530273,12877.669922,17401.912841,9658.252441,Let me tell you the truth. We have strong supp...
01-01-18,13921.530273,12877.669922,17401.912841,9658.252441,The Kumar Oscillator never fails...\r\n
01-01-18,13921.530273,12877.669922,17401.912841,9658.252441,"Get out while it’s not too crowded, the exit w..."


In [126]:

def add_placeholder(row):
    regex = r"\b[0-9]{3,}\b"
    if re.findall(regex,row['comment']):
        predictions_list = re.findall(regex,row['comment'])
        
        for prediction in predictions_list:    
            predictionVal = float(prediction.replace('$', ''))
    
            if predictionVal < row['Upper Bound'] and predictionVal > row['High']:
                return row['comment'].replace(prediction,'[OPTIMISTIC VALUE]')
            elif predictionVal > row['Lower Bound'] and predictionVal < row['Low']:
                return row['comment'].replace(prediction,'[PESSIMISTIC VALUE]')
            
    return row['comment']

def placeholder(df):
    df['updated_comment'] = df.apply(lambda x: add_placeholder(x), axis=1)
    
    return df

In [127]:
df_bc_merged = placeholder(df_bc_merged)
df_dj_merged = placeholder(df_dj_merged)
df_ts_merged = placeholder(df_ts_merged)

In [128]:
df_bc_merged.head(5)

,High,Low,Upper Bound,Lower Bound,comment,updated_comment
01-01-18,13921.530273,12877.669922,17401.912841,9658.252441,Drop now to 8600\r\n,Drop now to 8600\r\n
01-01-18,13921.530273,12877.669922,17401.912841,9658.252441,"If a panic kicks in, there is no support forge...","If a panic kicks in, there is no support forge..."
01-01-18,13921.530273,12877.669922,17401.912841,9658.252441,Let me tell you the truth. We have strong supp...,Let me tell you the truth. We have strong supp...
01-01-18,13921.530273,12877.669922,17401.912841,9658.252441,The Kumar Oscillator never fails...\r\n,The Kumar Oscillator never fails...\r\n
01-01-18,13921.530273,12877.669922,17401.912841,9658.252441,"Get out while it’s not too crowded, the exit w...","Get out while it’s not too crowded, the exit w..."


In [130]:
df_bc_merged.to_excel("data/output/bitcoin.xlsx", sheet_name='Sheet_1')
df_dj_merged.to_excel("data/output/dowjones.xlsx", sheet_name='Sheet_1')
df_ts_merged.to_excel("data/output/tesla.xlsx", sheet_name='Sheet_1')

/Users/morten/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:912: UserWarning: Ignoring URL 'https://invst.ly/9en6n.%20.%20I%20dont%20even%20need%20to%20draw%20lines%20on%20that%20chart%20for%20everyone%20to%20see%20what%20the%20trend%20is.%20if%20you%20see%20anything%20other%20than%20bearish%20then%20you%20are%20a%20mug.%20please%20reply%20to%20this%20post%20if%20you%20are%20a%20mug
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/morten/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:912: UserWarning: Ignoring URL 'https://www.cnbc.com/2018/01/04/trump-on-dow-25000-i-guess-our-new-number-is-30000.html%20Look%20what%20is%20Trump%20said%20today,%20so%20buy%20more%20stock,%20everybody%20can%20sell%20house%20and%20your%20family%20to%20buy%20more%20stock,%20make%20more%20easy%20money.
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
